<a href="https://colab.research.google.com/github/chiyeon01/ML_CONTEST/blob/main/DACON/%EC%B1%84%EB%AC%B4%20%EB%B6%88%EC%9D%B4%ED%96%89%20%EC%97%AC%EB%B6%80%20%EC%98%88%EC%B8%A1%20%ED%95%B4%EC%BB%A4%ED%86%A4%3A%20%EB%B6%88%EC%9D%B4%ED%96%89%EC%9D%98%20%EC%A7%95%ED%9B%84%EB%A5%BC%20%EC%B0%BE%EC%95%84%EB%9D%BC!/DecisionTree%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from google.colab import drive

In [149]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [150]:
!unzip '/content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/open.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/open.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ㅛ
error:  invalid response [ㅛ]
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [151]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
sample_df = pd.read_csv('/content/sample_submission.csv')

In [152]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  object 
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  object 
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  object 
 12  대출 상환 기간           10000 non-null  object 
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

In [153]:
train_df.head()

,UID,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,TRAIN_00000,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,TRAIN_00001,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,TRAIN_00002,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,TRAIN_00003,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,TRAIN_00004,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0


In [154]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                2062 non-null   object 
 1   주거 형태              2062 non-null   object 
 2   연간 소득              2062 non-null   float64
 3   현재 직장 근속 연수        2062 non-null   object 
 4   체납 세금 압류 횟수        2062 non-null   float64
 5   개설된 신용계좌 수         2062 non-null   int64  
 6   신용 거래 연수           2062 non-null   float64
 7   최대 신용한도            2062 non-null   float64
 8   신용 문제 발생 횟수        2062 non-null   int64  
 9   마지막 연체 이후 경과 개월 수  2062 non-null   int64  
 10  개인 파산 횟수           2062 non-null   int64  
 11  대출 목적              2062 non-null   object 
 12  대출 상환 기간           2062 non-null   object 
 13  현재 대출 잔액           2062 non-null   float64
 14  현재 미상환 신용액         2062 non-null   float64
 15  월 상환 부채액           2062 non-null   float64
 16  신용 점수              2062 

In [155]:
'''
train과 test 모두 null 값 없음.
UID는 ID 값이므로 삭제.
'''

'\ntrain과 test 모두 null 값 없음.\nUID는 ID 값이므로 삭제.\n'

In [156]:
data_df = pd.concat([train_df, test_df])

In [157]:
encoder = LabelEncoder()
object_columns = data_df.dtypes[data_df.dtypes == 'object'].index.tolist()

for column in object_columns:
    data_df[column] = encoder.fit_transform(data_df[column])

In [158]:
train_df = data_df[~data_df['채무 불이행 여부'].isnull()]
test_df = data_df[data_df['채무 불이행 여부'].isnull()]

In [159]:
train_df.shape, test_df.shape

((10000, 18), (2062, 18))

In [160]:
# features와 targets를 나누기.
features = train_df.iloc[:, :-1]
targets = train_df.iloc[:, -1]

In [161]:
features.drop(labels=['UID'], axis=1, inplace=True)

In [162]:
X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.15, stratify=targets, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(8500, 16) (1500, 16) (8500,) (1500,)


In [163]:
# 규제 없음
tree_clf1 = DecisionTreeClassifier()
tree_clf1.fit(X_train, y_train)

DecisionTreeClassifier()

In [164]:
# max_depth 규제
tree_clf2 = DecisionTreeClassifier(max_depth=3)
tree_clf2.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [165]:
# accuracy
print('규제를 적용하지 않은 모델의 train / val 점수: {0} / {1}'.format(tree_clf1.score(X_train, y_train), tree_clf1.score(X_val, y_val)))
print('규제를 적용한 모델의 train / val 점수: {0} / {1}'.format(tree_clf2.score(X_train, y_train), tree_clf2.score(X_val, y_val)))

규제를 적용하지 않은 모델의 train / val 점수: 1.0 / 0.6186666666666667
규제를 적용한 모델의 train / val 점수: 0.6854117647058824 / 0.6853333333333333


In [166]:
# roc_auc score
pred1 = tree_clf1.predict(X_val)
print(roc_auc_score(pred1, y_val))
pred2 = tree_clf2.predict(X_val)
print(roc_auc_score(pred2, y_val))

0.5815421475798834
0.6432


In [167]:
'''
규제를 적용하니 더 나아진 것으로 볼 수 있음.
하지만, 모든 상황에서 그런 것은 아님.
규제를 적용하지 않은 경우 과대적합이 일어남.

대회에서 적용하는 식(roc auc에서도 차이가 많이 남.)

lightgbm과 비교하면 0.03정도 차이가 나긴함.(확실히 차이가 꽤 남.)
'''

'\n규제를 적용하니 더 나아진 것으로 볼 수 있음.\n하지만, 모든 상황에서 그런 것은 아님.\n규제를 적용하지 않은 경우 과대적합이 일어남.\n\n대회에서 적용하는 식(roc auc에서도 차이가 많이 남.)\n\nlightgbm과 비교하면 0.03정도 차이가 나긴함.(확실히 차이가 꽤 남.)\n'

# 제출 파일 만들기.

In [168]:
# 전체 데이터셋에 대해 학습.
tree_clf2.fit(features, targets)

DecisionTreeClassifier(max_depth=3)

In [169]:
test_df.drop(labels='채무 불이행 여부', axis=1, inplace=True)

<ipython-input-169-94d585deaab1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(labels='채무 불이행 여부', axis=1, inplace=True)


In [170]:
pred = tree_clf2.predict_proba(test_df.iloc[:, 1:])

In [171]:
sample_df['채무 불이행 확률'] = pred[:, 1]

In [172]:
sample_df

,UID,채무 불이행 확률
0,TEST_0000,0.533973
1,TEST_0001,0.339314
2,TEST_0002,0.339314
3,TEST_0003,0.225318
4,TEST_0004,0.418109
...,...,...
2057,TEST_2057,0.074733
2058,TEST_2058,0.237462
2059,TEST_2059,0.418109
2060,TEST_2060,0.418109


In [173]:
sample_df.to_csv('/content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/Decision_Tree_pred.csv', index=False)